In [20]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    count
)
from pyspark.sql.functions import col, window, regexp_extract

In [2]:
input_dir="data/raw/*.parquet"
output_dir="data/reference"

In [3]:
spark = (
    SparkSession.builder.appName("Reddit sentiment Processing")
    .master("local[*]")
    .getOrCreate()
)

24/06/11 19:12:57 WARN Utils: Your hostname, Exporo-MBP-219.local resolves to a loopback address: 127.0.0.1; using 10.10.5.83 instead (on interface en0)
24/06/11 19:12:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/11 19:12:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/06/11 19:12:58 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
df = spark.read.parquet(input_dir)

In [13]:
user_ref = df.withColumn(
        "reference", regexp_extract(col("text"), r"/u/(\w+)", 0)
    ).filter(col("reference") != "")
post_ref = df.withColumn(
        "reference", regexp_extract(col("text"), r"/r/(\w+)", 0)
    ).filter(col("reference") != "")
url_ref = df.withColumn(
        "reference", regexp_extract(col("text"), r"(http\S+)", 0)
    ).filter(col("reference") != "")

In [14]:
combined_df = (
        user_ref.select("timestamp", "reference")
        .union(post_ref.select("timestamp", "reference"))
        .union(url_ref.select("timestamp", "reference"))
    )

In [23]:
windowed_counts = combined_df.groupBy(
        window(col("timestamp"), "60 seconds", "5 seconds"), col("reference")
    ).count()

In [24]:
windowed_counts.show(20)

+--------------------+--------------------+-----+
|              window|           reference|count|
+--------------------+--------------------+-----+
|{2024-06-05 08:37...|         /r/JoeRogan|    4|
|{2024-06-05 08:37...|         /r/JoeRogan|    4|
|{2024-06-05 08:37...|         /r/JoeRogan|    4|
|{2024-06-05 08:36...|         /r/JoeRogan|    4|
|{2024-06-05 08:37...|         /r/JoeRogan|    4|
|{2024-06-05 08:36...|         /r/JoeRogan|    4|
|{2024-06-05 08:37...|         /r/JoeRogan|    4|
|{2024-06-05 08:37...|         /r/JoeRogan|    4|
|{2024-06-05 08:37...|         /r/JoeRogan|    4|
|{2024-06-05 08:37...|         /r/JoeRogan|    4|
|{2024-06-05 08:37...|         /r/JoeRogan|    4|
|{2024-06-05 08:37...|         /r/JoeRogan|    4|
|{2024-06-05 00:05...|https://www.amazo...|    3|
|{2024-06-05 00:05...|https://www.amazo...|    3|
|{2024-06-05 00:06...|https://www.amazo...|    3|
|{2024-06-05 00:05...|https://www.amazo...|    3|
|{2024-06-05 00:05...|https://www.amazo...|    3|


In [ ]:
windowed_counts.write.mode("append").parquet(output_dir)

test

In [25]:
df_result = spark.read.parquet("data/reference/*.parquet")

In [26]:
df_result.show(20)

+--------------------+--------------------+-----+
|              window|           reference|count|
+--------------------+--------------------+-----+
|{2024-06-05 08:37...|         /r/JoeRogan|    4|
|{2024-06-05 08:37...|         /r/JoeRogan|    4|
|{2024-06-05 08:37...|         /r/JoeRogan|    4|
|{2024-06-05 08:36...|         /r/JoeRogan|    4|
|{2024-06-05 08:37...|         /r/JoeRogan|    4|
|{2024-06-05 08:36...|         /r/JoeRogan|    4|
|{2024-06-05 08:37...|         /r/JoeRogan|    4|
|{2024-06-05 08:37...|         /r/JoeRogan|    4|
|{2024-06-05 08:37...|         /r/JoeRogan|    4|
|{2024-06-05 08:37...|         /r/JoeRogan|    4|
|{2024-06-05 08:37...|         /r/JoeRogan|    4|
|{2024-06-05 08:37...|         /r/JoeRogan|    4|
|{2024-06-05 00:05...|https://www.amazo...|    3|
|{2024-06-05 00:05...|https://www.amazo...|    3|
|{2024-06-05 00:06...|https://www.amazo...|    3|
|{2024-06-05 00:05...|https://www.amazo...|    3|
|{2024-06-05 00:05...|https://www.amazo...|    3|
